# Week 3 - Part 2

This notebook contains the second submission

In [39]:
print("This notebook contains the second submission")

This notebook contains the second submission


In [44]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from unicodedata import normalize
import requests
import html5lib
import lxml
import xlrd
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

In [45]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'html5lib')

postal_codes_dict = {} # initialize an empty dictionary to save the data in

In [46]:
for table_cell in soup.find_all('td'):
    try:
        postal_code = table_cell.p.b.text # get the postal code
        postal_code_investigate = table_cell.span.text
        neighborhoods_data = table_cell.span.text # get the rest of the data in the cell
        borough = neighborhoods_data.split('(')[0] # get the borough in the cell
        
        # if the cell is not assigned then ignore it
        if neighborhoods_data == 'Not assigned':
            neighborhoods = []
        # else process the data and add it to the dictionary
        else:
            postal_codes_dict[postal_code] = {}
            
            try:
                neighborhoods = neighborhoods_data.split('(')[1]
            
                # remove parantheses from neighborhoods string
                neighborhoods = neighborhoods.replace('(', ' ')
                neighborhoods = neighborhoods.replace(')', ' ')

                neighborhoods_names = neighborhoods.split('/')
                neighborhoods_clean = ', '.join([name.strip() for name in neighborhoods_names])
            except:
                borough = borough.strip('\n')
                neighborhoods_clean = borough
 
            # add borough and neighborhood to dictionary
            postal_codes_dict[postal_code]['borough'] = borough
            postal_codes_dict[postal_code]['neighborhoods'] = neighborhoods_clean
    except:
        pass

In [47]:
# create an empty dataframe
columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(columns=columns)
toronto_data

,PostalCode,Borough,Neighborhood


In [48]:
# populate dataframe with data from dictionary
for ind, postal_code in enumerate(postal_codes_dict):
    borough = postal_codes_dict[postal_code]['borough']
    neighborhood = postal_codes_dict[postal_code]['neighborhoods']
    toronto_data = toronto_data.append({"PostalCode": postal_code, 
                                        "Borough": borough, 
                                        "Neighborhood": neighborhood},
                                        ignore_index=True)

# print number of rows of dataframe
toronto_data.shape[0]

103

In [56]:
toronto_data_cor = pd.read_csv("Geospatial_Coordinates.csv") 
toronto_data_cor.head(5)

,Latitude,Longitude
PostalCode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [66]:
df = pd.merge(toronto_data, data, on='PostalCode')
df.head(5)

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M5N,Central Toronto,Roselawn,43.711695,-79.416936
M4P,Central Toronto,Davisville North,43.712751,-79.390197
M5P,Central Toronto,Forest Hill North & West Forest Hill Road Park,43.696948,-79.411307
M4R,Central Toronto,North Toronto West Lawrence Park,43.715383,-79.405678
M5R,Central Toronto,The Annex North Midtown Yorkville,43.672710,-79.405678
